In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from elftools.elf.elffile import ELFFile


class Sections:
    def __init__(self, path: Path):
        with open(path, "rb") as fin:
            self.data = self.get_section_info(ELFFile(fin))

    @staticmethod
    def get_section_info(elffile: ELFFile):
        import pandas as pd

        df = pd.DataFrame(
            [
                {
                    "name": s.name,
                    **{
                        k.removeprefix("sh_"): v
                        for k, v in s.header.items()
                        if k not in ("sh_name", "sh_addr")
                    },
                    "addr": s.header.sh_addr,
                    # "data": s.data()[:15],
                }
                for s in elffile.iter_sections()
            ]
        ).set_index("name")
        return df

    def _repr_html_(self):
        return self.data.to_html(formatters={"addr": hex, "offset": hex, "size": hex})


In [4]:
from depsurf import VERSION_DEFAULT

Sections(VERSION_DEFAULT.vmlinux_path)

,type,flags,offset,size,link,info,addralign,entsize,addr
name,,,,,,,,,
,SHT_NULL,0,0x0,0x0,0,0,0,0,0x0
.text,SHT_PROGBITS,6,0x200000,0xe00eb1,0,0,4096,0,0xffffffff81000000
.rela.text,SHT_RELA,64,0x15847850,0x86c718,72,1,8,24,0x0
.notes,SHT_NOTE,2,0x1000eb4,0x1ec,0,0,4,0,0xffffffff81e00eb4
.rela.notes,SHT_RELA,64,0x160b3f68,0x48,72,3,8,24,0x0
__ex_table,SHT_PROGBITS,2,0x10010a0,0x82a4,0,0,4,0,0xffffffff81e010a0
.rela__ex_table,SHT_RELA,64,0x160b3fb0,0x30fd8,72,5,8,24,0x0
.rodata,SHT_PROGBITS,3,0x1200000,0x477bb2,0,0,4096,0,0xffffffff82000000
.rela.rodata,SHT_RELA,64,0x160e4f88,0x10eee8,72,7,8,24,0x0
